In [22]:
# mnist数据集下载，下载之后的文件在MNIST_data文件夹下
from tensorflow.examples.tutorials.mnist import input_data
 
data = input_data.read_data_sets('MNIST_data/')

# 定义load_data()函数以读取数据
def load_data(data_path):
    '''
    函数功能：导出MNIST数据
    输入: data_path   传入数据所在路径（解压后的数据）
    输出: train_data  输出data，形状为(60000, 28, 28, 1)    
         train_label  输出label，形状为(60000, 1)                 
    '''
 
    f_data = open(os.path.join(data_path, 'train-images.idx3-ubyte'))
    loaded_data = np.fromfile(file=f_data, dtype=np.uint8)
    #前16个字符为说明符，需要跳过
    train_data = loaded_data[16:].reshape((-1, 784)).astype(np.float)
 
    f_label = open(os.path.join(data_path, 'train-labels.idx1-ubyte'))
    loaded_label = np.fromfile(file=f_label, dtype=np.uint8)
    #前8个字符为说明符，需要跳过
    train_label = loaded_label[8:].reshape((-1)).astype(np.float)
 
    return train_data, train_label

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [32]:
# 导入需要的包
import os                         #读取路径下文件
import shutil                     #递归删除文件
import tensorflow as tf           #编写神经网络
import numpy as np                #矩阵运算操作
# from skimage.io import imsave     #保存影像
import imageio
imsave = imageio.imsave
from imageio import imsave 
from tensorflow.examples.tutorials.mnist import input_data  #第一次下载数据时用
 
# 图像的size为(28, 28, 1)
image_height = 28           
image_width = 28
image_size = image_height * image_width
 
# 是否训练和存储设置
train = True
restore = False              #是否存储训练结果
output_path = "./output/"    #存储文件的路径
 
# 实验所需的超参数
max_epoch = 500
batch_size = 256
h1_size = 256         #第一隐藏层的size，即特征数
h2_size = 512         #第二隐藏层的size，即特征数
z_size = 128          #生成器的传入参数

In [33]:
#导入tensorflow
import tensorflow as tf
 
#定义GAN的生成器
def generator(z_prior):
    '''
    函数功能：生成影像，参与训练过程
    输入：z_prior,       #输入tf格式，size为（batch_size, z_size）的数据
    输出：x_generate,    #生成图像
         g_params,      #生成图像的所有参数
    '''
    # 第一个链接层 
    #以2倍标准差stddev的截断的正态分布中生成大小为[z_size, h1_size]的随机值，权值weight初始化。
    w1 = tf.Variable(tf.truncated_normal([z_size, h1_size], stddev=0.1), name="g_w1", dtype=tf.float32)
    #生成大小为[h1_size]的0值矩阵，偏置bias初始化
    b1 = tf.Variable(tf.zeros([h1_size]), name="g_b1", dtype=tf.float32)
    #通过矩阵运算，将输入z_prior传入隐含层h1。激活函数为relu
    h1 = tf.nn.relu(tf.matmul(z_prior, w1) + b1)
 
    # 第二个链接层 
    #以2倍标准差stddev的截断的正态分布中生成大小为[h1_size, h2_size]的随机值，权值weight初始化。
    w2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=0.1), name="g_w2", dtype=tf.float32)
    #生成大小为[h2_size]的0值矩阵，偏置bias初始化
    b2 = tf.Variable(tf.zeros([h2_size]), name="g_b2", dtype=tf.float32)
    #通过矩阵运算，将h1传入隐含层h2。激活函数为relu
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
 
    # 第三个链接层 
    #以2倍标准差stddev的截断的正态分布中生成大小为[h2_size, image_size]的随机值，权值weight初始化。
    w3 = tf.Variable(tf.truncated_normal([h2_size, image_size], stddev=0.1), name="g_w3", dtype=tf.float32)
    #生成大小为[image_size]的0值矩阵，偏置bias初始化
    b3 = tf.Variable(tf.zeros([image_size]), name="g_b3", dtype=tf.float32)
    #通过矩阵运算，将h2传入隐含层h3。
    h3 = tf.matmul(h2, w3) + b3
    #利用tanh激活函数，将h3传入输出层
    x_generate = tf.nn.tanh(h3)
 
    #将所有参数合并到一起
    g_params = [w1, b1, w2, b2, w3, b3]
 
    return x_generate, g_params

In [34]:
# 定义GAN的判别器 
def discriminator(x_data, x_generated, keep_prob):
    '''
    函数功能：对输入数据进行判断，并保存其参数
    输入：x_data,        #输入的真实数据 
        x_generated,     #生成器生成的虚假数据
        keep_prob，      #dropout率，防止过拟合
    输出：y_data,        #判别器对batch个数据的处理结果
        y_generated,     #判别器对余下数据的处理结果
        d_params，       #判别器的参数
    '''
 
    # 合并输入数据，包括真实数据x_data和生成器生成的假数据x_generated
    x_in = tf.concat([x_data, x_generated], 0) 
 
    # 第一个链接层
    #以2倍标准差stddev的截断的正态分布中生成大小为[image_size, h2_size]的随机值，权值weight初始化。
    w1 = tf.Variable(tf.truncated_normal([image_size, h2_size], stddev=0.1), name="d_w1", dtype=tf.float32)
    #生成大小为[h2_size]的0值矩阵，偏置bias初始化
    b1 = tf.Variable(tf.zeros([h2_size]), name="d_b1", dtype=tf.float32)
    #通过矩阵运算，将输入x_in传入隐含层h1.同时以一定的dropout率舍弃节点，防止过拟合
    h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(x_in, w1) + b1), keep_prob)
 
    # 第二个链接层
    #以2倍标准差stddev的截断的正态分布中生成大小为[h2_size, h1_size]的随机值，权值weight初始化。
    w2 = tf.Variable(tf.truncated_normal([h2_size, h1_size], stddev=0.1), name="d_w2", dtype=tf.float32)
    #生成大小为[h1_size]的0值矩阵，偏置bias初始化
    b2 = tf.Variable(tf.zeros([h1_size]), name="d_b2", dtype=tf.float32)
    #通过矩阵运算，将h1传入隐含层h2.同时以一定的dropout率舍弃节点，防止过拟合
    h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h1, w2) + b2), keep_prob)
 
    # 第三个链接层
    #以2倍标准差stddev的截断的正态分布中生成大小为[h1_size, 1]的随机值，权值weight初始化。
    w3 = tf.Variable(tf.truncated_normal([h1_size, 1], stddev=0.1), name="d_w3", dtype=tf.float32)
    #生成0值，偏置bias初始化
    b3 = tf.Variable(tf.zeros([1]), name="d_b3", dtype=tf.float32)
    #通过矩阵运算，将h2传入隐含层h3
    h3 = tf.matmul(h2, w3) + b3
 
    #从h3中切出batch_size张图像
    y_data = tf.nn.sigmoid(tf.slice(h3, [0, 0], [batch_size, -1], name=None))
    #从h3中切除余下的图像
    y_generated = tf.nn.sigmoid(tf.slice(h3, [batch_size, 0], [-1, -1], name=None))
 
    #判别器的所有参数
    d_params = [w1, b1, w2, b2, w3, b3]
 
    return y_data, y_generated, d_params

In [35]:
#显示结果的函数
def show_result(batch_res, fname, grid_size=(8, 8), grid_pad=5):
    '''
    函数功能：输入相关参数，将运行结果以图片的形式保存到当前路径下
    输入：batch_res,       #输入数据
        fname,             #输入路径
        grid_size=(8, 8),  #默认输出图像为8*8张
        grid_pad=5，       #默认图像的边缘留白为5像素
    输出：无
    '''
    
    #将batch_res进行值[0, 1]归一化，同时将其reshape成（batch_size, image_height, image_width）
    batch_res = 0.5 * batch_res.reshape((batch_res.shape[0], image_height, image_width)) + 0.5
    #重构显示图像格网的参数
    img_h, img_w = batch_res.shape[1], batch_res.shape[2]
    grid_h = img_h * grid_size[0] + grid_pad * (grid_size[0] - 1)
    grid_w = img_w * grid_size[1] + grid_pad * (grid_size[1] - 1)
    img_grid = np.zeros((grid_h, grid_w), dtype=np.uint8)
    for i, res in enumerate(batch_res):
        if i >= grid_size[0] * grid_size[1]:
            break
        img = (res) * 255.
        img = img.astype(np.uint8)
        row = (i // grid_size[0]) * (img_h + grid_pad)
        col = (i % grid_size[1]) * (img_w + grid_pad)
        img_grid[row:row + img_h, col:col + img_w] = img
    #保存图像
    imsave(fname, img_grid)

In [36]:
# 定义训练过程
def train():
    '''
    函数功能：训练整个GAN网络，并随机生成手写数字
    输入：无
    输出：sess.saver()
    '''
 
    # 加载数据
    train_data, train_label = load_data("MNIST_data")
    size = train_data.shape[0]
 
    # 构建模型---------------------------------------------------------------------
    # 定义GAN网络的输入，其中x_data为[batch_size, image_size], z_prior为[batch_size, z_size]
    x_data = tf.placeholder(tf.float32, [batch_size, image_size], name="x_data") # (batch_size, image_size)
    z_prior = tf.placeholder(tf.float32, [batch_size, z_size], name="z_prior") # (batch_size, z_size)
    # 定义dropout率
    keep_prob = tf.placeholder(tf.float32, name="keep_prob") 
    global_step = tf.Variable(0, name="global_step", trainable=False)
 
    # 利用生成器生成数据x_generated和参数g_params
    x_generated, g_params = generator(z_prior)
    # 利用判别器判别生成器的结果
    y_data, y_generated, d_params = discriminator(x_data, x_generated, keep_prob)
 
    # 定义判别器和生成器的loss函数
    d_loss = - (tf.log(y_data) + tf.log(1 - y_generated))
    g_loss = - tf.log(y_generated)
 
    # 设置学习率为0.0001，用AdamOptimizer进行优化
    optimizer = tf.train.AdamOptimizer(0.0001)
 
    # 判别器discriminator 和生成器 generator 对损失函数进行最小化处理
    d_trainer = optimizer.minimize(d_loss, var_list=d_params)
    g_trainer = optimizer.minimize(g_loss, var_list=g_params)
    # 模型构建完毕--------------------------------------------------------------------
 
    # 全局变量初始化
    init = tf.global_variables_initializer()
 
    # 启动会话sess
    saver = tf.train.Saver()
    sess = tf.Session()
    sess.run(init)
 
    # 判断是否需要存储
    if restore:
        #若是，将最近一次的checkpoint点存到outpath下
        chkpt_fname = tf.train.latest_checkpoint(output_path)
        saver.restore(sess, chkpt_fname)
    else:
        #若否，判断目录是存在，如果目录存在，则递归的删除目录下的所有内容，并重新建立目录
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.mkdir(output_path)
 
    # 利用随机正态分布产生噪声影像，尺寸为(batch_size, z_size)
    z_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
 
    # 逐个epoch内训练
    for i in range(sess.run(global_step), max_epoch):
        # 图像每个epoch内可以放(size // batch_size)个size
        for j in range(size // batch_size):
            if j%20 == 0:
                print("epoch:%s, iter:%s" % (i, j))
            
            # 训练一个batch的数据
            batch_end = j * batch_size + batch_size
            if batch_end >= size:
                batch_end = size - 1
            x_value = train_data[ j * batch_size : batch_end ]
            # 将数据归一化到[-1, 1]
            x_value = x_value / 255.
            x_value = 2 * x_value - 1
            
            # 以正太分布的形式产生随机噪声
            z_value = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
            # 每个batch下，输入数据运行GAN，训练判别器
            sess.run(d_trainer,
                     feed_dict={x_data: x_value, z_prior: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
            # 每个batch下，输入数据运行GAN，训练生成器
            if j % 1 == 0:
                sess.run(g_trainer,
                         feed_dict={x_data: x_value, z_prior: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
        # 每一个epoch中的所有batch训练完后，利用z_sample测试训练后的生成器
        x_gen_val = sess.run(x_generated, feed_dict={z_prior: z_sample_val})
        # 每一个epoch中的所有batch训练完后，显示生成器的结果，并打印生成结果的值
        show_result(x_gen_val, os.path.join(output_path, "sample%s.jpg" % i))
        print(x_gen_val)
        # 每一个epoch中，生成随机分布以重置z_random_sample_val
        z_random_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
        # 每一个epoch中，利用z_random_sample_val生成手写数字图像，并显示结果
        x_gen_val = sess.run(x_generated, feed_dict={z_prior: z_random_sample_val})
        show_result(x_gen_val, os.path.join(output_path, "random_sample%s.jpg" % i))
        # 保存会话
        sess.run(tf.assign(global_step, i + 1))
        saver.save(sess, os.path.join(output_path, "model"), global_step=global_step)

In [37]:
if __name__ == '__main__':
    if train:
        train()

epoch:0, iter:0
epoch:0, iter:20
epoch:0, iter:40
epoch:0, iter:60
epoch:0, iter:80
epoch:0, iter:100
epoch:0, iter:120
epoch:0, iter:140
epoch:0, iter:160
epoch:0, iter:180
epoch:0, iter:200
epoch:0, iter:220
[[-0.69190359 -0.99999833 -0.59808093 ..., -0.99887443 -0.99978679
  -0.99901134]
 [-0.91656065 -0.99999863 -0.49715534 ..., -0.15644895 -0.99965423
  -0.94964951]
 [ 0.28955734 -0.99995446 -0.8062402  ..., -0.99962544 -0.99974519
  -0.99964863]
 ..., 
 [ 0.5231483  -0.9999994  -0.8951295  ..., -0.99105233 -0.99999642
  -0.99994731]
 [-0.74749064 -0.99999952 -0.57163894 ..., -0.99911231 -0.99786657
  -0.99671662]
 [-0.71061265 -0.99984658 -0.6028859  ..., -0.99583644 -0.99995738
  -0.99978089]]
epoch:1, iter:0
epoch:1, iter:20
epoch:1, iter:40
epoch:1, iter:60
epoch:1, iter:80
epoch:1, iter:100
epoch:1, iter:120
epoch:1, iter:140
epoch:1, iter:160
epoch:1, iter:180
epoch:1, iter:200
epoch:1, iter:220
[[-0.99493104  0.99802673 -0.99886966 ..., -0.99990624  0.83209503
  -0.99957764

epoch:12, iter:0
epoch:12, iter:20
epoch:12, iter:40
epoch:12, iter:60
epoch:12, iter:80
epoch:12, iter:100
epoch:12, iter:120
epoch:12, iter:140
epoch:12, iter:160
epoch:12, iter:180
epoch:12, iter:200
epoch:12, iter:220
[[-0.99997526 -0.99999821 -0.9999854  ...,  0.99961376 -0.99986756
  -0.9999997 ]
 [-0.99995387 -0.99999863 -0.99894148 ...,  0.99977171 -0.99963832
  -0.99990934]
 [-0.99981874 -0.99999291 -0.99947244 ...,  0.99213082 -0.99993348
  -0.99999928]
 ..., 
 [-0.99988908 -0.99999183 -0.99988145 ...,  0.99994004 -0.99999851
  -0.99999982]
 [-0.99991035 -0.99958473 -0.99841535 ...,  0.92932528 -0.99982125
  -0.99981165]
 [-0.99679583 -0.99460071 -0.99535125 ...,  0.99503106 -0.99874008
  -0.99992973]]
epoch:13, iter:0
epoch:13, iter:20
epoch:13, iter:40
epoch:13, iter:60
epoch:13, iter:80
epoch:13, iter:100
epoch:13, iter:120
epoch:13, iter:140
epoch:13, iter:160
epoch:13, iter:180
epoch:13, iter:200
epoch:13, iter:220
[[-0.99999988 -0.99999893 -0.99999964 ..., -0.99999082 -

epoch:23, iter:100
epoch:23, iter:120
epoch:23, iter:140
epoch:23, iter:160
epoch:23, iter:180
epoch:23, iter:200
epoch:23, iter:220
[[-0.99336135 -0.99926317 -0.99883342 ..., -0.99921322 -0.99532551
  -0.9901759 ]
 [-0.916457   -0.30047497 -0.59389162 ..., -0.46219578 -0.9701615
  -0.72353244]
 [-0.98807728 -0.9580729  -0.97523075 ..., -0.99527007 -0.99759954
  -0.99264401]
 ..., 
 [-0.92238933 -0.55376887 -0.79420477 ..., -0.95981878 -0.94945061
  -0.96621478]
 [-0.83235461  0.3441284  -0.61703295 ..., -0.85302609 -0.62992877
  -0.72967994]
 [-0.44018927 -0.23687597 -0.54659134 ..., -0.17868261 -0.31115556
  -0.50953472]]
epoch:24, iter:0
epoch:24, iter:20
epoch:24, iter:40
epoch:24, iter:60
epoch:24, iter:80
epoch:24, iter:100
epoch:24, iter:120
epoch:24, iter:140
epoch:24, iter:160
epoch:24, iter:180
epoch:24, iter:200
epoch:24, iter:220
[[-0.62779206 -0.99985343 -0.99953276 ..., -0.99778253 -0.99898279
  -0.67840487]
 [-0.7098434  -0.72605753 -0.89472646 ..., -0.29540113 -0.991246

epoch:35, iter:0
epoch:35, iter:20
epoch:35, iter:40
epoch:35, iter:60
epoch:35, iter:80
epoch:35, iter:100
epoch:35, iter:120
epoch:35, iter:140
epoch:35, iter:160
epoch:35, iter:180
epoch:35, iter:200
epoch:35, iter:220
[[-0.99997127 -0.99999917 -0.98752463 ..., -0.63209254 -0.99999028
  -0.99998063]
 [-0.97538024 -0.97358012 -0.99363792 ..., -0.93442857 -0.99532896
  -0.899441  ]
 [-0.99998724 -0.99961996 -0.99442685 ..., -0.89990658 -0.99993241
  -0.99749684]
 ..., 
 [-0.99490249 -0.99284422 -0.97928774 ..., -0.9592461  -0.99980378
  -0.99181151]
 [-0.90392327 -0.86983979 -0.71938258 ..., -0.96830416 -0.95059586
  -0.97804672]
 [-0.7366116  -0.80113244 -0.47233036 ..., -0.2396048  -0.597045
  -0.67709398]]
epoch:36, iter:0
epoch:36, iter:20
epoch:36, iter:40
epoch:36, iter:60
epoch:36, iter:80
epoch:36, iter:100
epoch:36, iter:120
epoch:36, iter:140
epoch:36, iter:160
epoch:36, iter:180
epoch:36, iter:200
epoch:36, iter:220
[[-0.99999046 -0.99999982 -0.99860096 ..., -0.98586339 -0.

epoch:47, iter:0
epoch:47, iter:20
epoch:47, iter:40
epoch:47, iter:60
epoch:47, iter:80
epoch:47, iter:100
epoch:47, iter:120
epoch:47, iter:140
epoch:47, iter:160
epoch:47, iter:180
epoch:47, iter:200
epoch:47, iter:220
[[-0.99987757 -0.99990159 -0.99999952 ..., -0.9996829  -0.9999842
  -0.99999964]
 [-0.84662825 -0.99478662 -0.97199303 ..., -0.92953002 -0.20916165
  -0.98308915]
 [-0.99928188 -0.99970913 -0.99997169 ..., -0.99864185 -0.98998153
  -0.99995363]
 ..., 
 [-0.94371969 -0.99123406 -0.99561262 ..., -0.99728799 -0.98243856
  -0.99885935]
 [-0.95276779 -0.99585879 -0.85026348 ..., -0.99427861 -0.07425708
  -0.99932641]
 [-0.95304239 -0.7596308  -0.84536898 ..., -0.36222589 -0.90067279
  -0.9233492 ]]
epoch:48, iter:0
epoch:48, iter:20
epoch:48, iter:40
epoch:48, iter:60
epoch:48, iter:80
epoch:48, iter:100
epoch:48, iter:120
epoch:48, iter:140
epoch:48, iter:160
epoch:48, iter:180
epoch:48, iter:200
epoch:48, iter:220
[[-0.9999997  -0.99991548 -0.99999863 ..., -0.99999195 -0

...

epoch:451, iter:0
epoch:451, iter:20
epoch:451, iter:40
epoch:451, iter:60
epoch:451, iter:80
epoch:451, iter:100
epoch:451, iter:120
epoch:451, iter:140
epoch:451, iter:160
epoch:451, iter:180
epoch:451, iter:200
epoch:451, iter:220
[[-0.99945629 -0.99992615 -0.99993932 ..., -0.9885034  -0.99231076
  -0.99773628]
 [-0.99546754 -0.96680808 -0.99541652 ..., -0.99988365 -0.99652869
  -0.98739821]
 [-0.98197305 -0.99643272 -0.65385872 ..., -0.98527926 -0.99610734
  -0.99429691]
 ..., 
 [-0.99405259 -0.99997091 -0.9997102  ..., -0.99974746 -0.99892396
  -0.99567753]
 [-0.99159771 -0.99885762 -0.98111826 ..., -0.99548137 -0.99712479
  -0.99627471]
 [-0.99366939 -0.97222793 -0.99965417 ..., -0.9699868  -0.99283516
  -0.9900282 ]]
epoch:452, iter:0
epoch:452, iter:20
epoch:452, iter:40
epoch:452, iter:60
epoch:452, iter:80
epoch:452, iter:100
epoch:452, iter:120
epoch:452, iter:140
epoch:452, iter:160
epoch:452, iter:180
epoch:452, iter:200
epoch:452, iter:220
[[-0.99952227 -0.99997169 -0.999

epoch:462, iter:140
epoch:462, iter:160
epoch:462, iter:180
epoch:462, iter:200
epoch:462, iter:220
[[-0.99960619 -0.99997759 -0.9999842  ..., -0.99460858 -0.99466211
  -0.99893272]
 [-0.9926731  -0.97912574 -0.99133009 ..., -0.99989527 -0.99673337
  -0.98745322]
 [-0.97389609 -0.99604595 -0.56227654 ..., -0.9942624  -0.99554473
  -0.98990703]
 ..., 
 [-0.98932278 -0.99999601 -0.99984336 ..., -0.99992645 -0.99973589
  -0.99878204]
 [-0.98335963 -0.99976283 -0.98581219 ..., -0.99921542 -0.9995724
  -0.99941587]
 [-0.98416948 -0.98390615 -0.99956149 ..., -0.96161503 -0.9983027
  -0.99602407]]
epoch:463, iter:0
epoch:463, iter:20
epoch:463, iter:40
epoch:463, iter:60
epoch:463, iter:80
epoch:463, iter:100
epoch:463, iter:120
epoch:463, iter:140
epoch:463, iter:160
epoch:463, iter:180
epoch:463, iter:200
epoch:463, iter:220
[[-0.99980772 -0.99992675 -0.99999166 ..., -0.99377924 -0.99019533
  -0.99905241]
 [-0.99464113 -0.97450572 -0.99490076 ..., -0.99988252 -0.99426693
  -0.96515268]
 [-0

epoch:474, iter:0
epoch:474, iter:20
epoch:474, iter:40
epoch:474, iter:60
epoch:474, iter:80
epoch:474, iter:100
epoch:474, iter:120
epoch:474, iter:140
epoch:474, iter:160
epoch:474, iter:180
epoch:474, iter:200
epoch:474, iter:220
[[-0.99841595 -0.99989372 -0.99984795 ..., -0.96730673 -0.98036987
  -0.99895424]
 [-0.99548274 -0.98297185 -0.99013239 ..., -0.99982375 -0.99165297
  -0.97401017]
 [-0.97838396 -0.99798936 -0.64216316 ..., -0.98459649 -0.98870647
  -0.99507207]
 ..., 
 [-0.99300277 -0.99999362 -0.99981374 ..., -0.99987233 -0.99822223
  -0.99794233]
 [-0.99458635 -0.99967897 -0.99369407 ..., -0.9985249  -0.99847263
  -0.99939954]
 [-0.9747839  -0.98889238 -0.99929285 ..., -0.87619501 -0.99732852
  -0.99781036]]
epoch:475, iter:0
epoch:475, iter:20
epoch:475, iter:40
epoch:475, iter:60
epoch:475, iter:80
epoch:475, iter:100
epoch:475, iter:120
epoch:475, iter:140
epoch:475, iter:160
epoch:475, iter:180
epoch:475, iter:200
epoch:475, iter:220
[[-0.9962061  -0.99991906 -0.999

epoch:485, iter:140
epoch:485, iter:160
epoch:485, iter:180
epoch:485, iter:200
epoch:485, iter:220
[[-0.99853516 -0.99988681 -0.99985188 ..., -0.9740786  -0.99362695
  -0.99801081]
 [-0.99216485 -0.94383752 -0.98128468 ..., -0.99975216 -0.99092722
  -0.96535277]
 [-0.99189764 -0.99570751 -0.76163435 ..., -0.98998803 -0.99670064
  -0.99615782]
 ..., 
 [-0.99664575 -0.99999696 -0.99989194 ..., -0.99992436 -0.9996177
  -0.99935013]
 [-0.99717832 -0.99915248 -0.99703532 ..., -0.99715233 -0.99963117
  -0.99931473]
 [-0.96801364 -0.98679906 -0.99738991 ..., -0.89437407 -0.99961662
  -0.99726552]]
epoch:486, iter:0
epoch:486, iter:20
epoch:486, iter:40
epoch:486, iter:60
epoch:486, iter:80
epoch:486, iter:100
epoch:486, iter:120
epoch:486, iter:140
epoch:486, iter:160
epoch:486, iter:180
epoch:486, iter:200
epoch:486, iter:220
[[-0.99882376 -0.99990469 -0.99975276 ..., -0.9907136  -0.99291593
  -0.99879223]
 [-0.98887378 -0.94734168 -0.96990901 ..., -0.9998281  -0.99169302
  -0.97063857]
 [-

epoch:497, iter:0
epoch:497, iter:20
epoch:497, iter:40
epoch:497, iter:60
epoch:497, iter:80
epoch:497, iter:100
epoch:497, iter:120
epoch:497, iter:140
epoch:497, iter:160
epoch:497, iter:180
epoch:497, iter:200
epoch:497, iter:220
[[-0.99857074 -0.99995279 -0.99994862 ..., -0.96011209 -0.99283653
  -0.99896914]
 [-0.99637955 -0.99110925 -0.97674227 ..., -0.99914092 -0.99012691
  -0.97038466]
 [-0.99234539 -0.9965803  -0.88837647 ..., -0.97872651 -0.99715096
  -0.9976837 ]
 ..., 
 [-0.99748468 -0.99999672 -0.99997747 ..., -0.99984145 -0.99946922
  -0.99933821]
 [-0.99740773 -0.99954188 -0.99889565 ..., -0.98546356 -0.99967003
  -0.99898922]
 [-0.96772426 -0.98541647 -0.99878561 ..., -0.64273566 -0.99938142
  -0.99566579]]
epoch:498, iter:0
epoch:498, iter:20
epoch:498, iter:40
epoch:498, iter:60
epoch:498, iter:80
epoch:498, iter:100
epoch:498, iter:120
epoch:498, iter:140
epoch:498, iter:160
epoch:498, iter:180
epoch:498, iter:200
epoch:498, iter:220
[[-0.99878258 -0.99995404 -0.999